In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import GPy
import glob
import os
import tensorflow as tf
from warnings import catch_warnings

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import sys
import scipy
import math

tfk = tf.keras
from sklearn import model_selection
from keras.models import Sequential
from keras import optimizers
from keras.layers import  Input, Dense
from keras.models import Model
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from warnings import catch_warnings
from warnings import simplefilter
from scipy.stats import norm


seed_value = 42




In [ ]:
def normalization(a):
    # the +1 is to normalize in [1 2] as otherwise the relative error would encounter division by 0
    return (a - a.min(axis = 0))/(a.max(axis = 0) - a.min(axis = 0)) 

def customLoss(y_actual,y_pred): 
    custom_loss = tf.math.divide( abs(y_pred-y_actual) , y_actual)
    custom_loss=tf.reduce_mean(custom_loss)
    return custom_loss

def rmse(predictions, targets):
    return np.sqrt(((predictions.flatten() - targets.flatten()) ** 2).mean())

def model_evaluation(model,X_test,y_test):
    predictions_film = model.predict(X_test)

    idx_test_film = np.argmax(predictions_film, axis=-1)
    idx_test_t_film = np.argmax(y_test, axis=-1)
    error_film = idx_test_film - idx_test_t_film
    print(100 - 100*np.count_nonzero(error_film)/error_film.shape[0])
    plt.hist(idx_test_film - idx_test_t_film )
    plt.show()

    cnf_matrix = confusion_matrix(np.argmax(predictions_film, axis=-1),np.argmax(y_test, axis=-1))
    print(cnf_matrix)

In [3]:
k = os.path.abspath("data") 
parameters = pd.read_csv(k + '/' +'Compositions'+'.csv')
pbr2 = np.array((parameters['PbBr2  (mL)'].astype(float)))
oleate = np.array((parameters['Cs-oleate (mL)'].astype(float)))
aceton = np.array((parameters['Aceton (mL)'].astype(float)))

pbr2 = parameters['PbBr2  (mL)']
oleate = parameters['Cs-oleate (mL)']
aceton = parameters['Aceton (mL)']

aceton = aceton
inputs = np.array([pbr2,oleate,aceton]).T
file_nr = len(glob.glob(k + '/'+"*.csv"))
file_nr

221

# Loading the Data

In [4]:
mean_median_metric = []
hwhm = []
max_peak = []
for i in range(1,file_nr): 
    a = pd.read_csv(k + '/' + str(i)+'.csv')
    a = a.iloc[1:]
    d = np.array((a['S1c / R1c'].astype(float)))
    w = np.array(a['Wavelength'].astype(float))
    d = np.reshape(d, (d.shape[0],1))
    w = np.reshape(w, (w.shape[0],1))
    
    
    d_max = np.max(d)
    d = d/d_max
    max_index = np.argmax(d, axis=0)
    
    
    init = 0
    ending = d.shape[0]
    for o in range(0,max_index[0]):
        if d[o] >0.05:
            init = o
            break
    for o in range(max_index[0], d.shape[0]):
        if d[o]<0.05:
            ending = o
            break
    



    max_index = np.argmax(d, axis=0)
    
    data_range = min((max_index), w.shape[0]-int(max_index))
    
    
    d_left = d[int(init):int(max_index)]
    w_left = w[int(init):int(max_index)]

    d_right = d[int(max_index): int(ending)]
    w_right = w[int(max_index): int(ending)]
    d_right = d_right[::-1]
    w_right = w_right[::-1]

    mean = w[np.argmax(d)]
    median_cumsum = np.cumsum(d)
    index = np.argmax(median_cumsum>median_cumsum[-1]*0.5)
    median = w[index]

    mean_median_metric_sample = 100*abs((mean-median)/mean)
    mean_median_metric.append(mean_median_metric_sample)

    first_half = np.argwhere(np.array(d) >  0.5*np.max(d))[0][0] - 1
    second_half = np.argwhere(d[np.argmax(d):] <  0.5*np.max(d))[0][0] + np.argmax(d)-1

    hwhm_sample =abs( w[first_half] - w[second_half])
    hwhm.append(hwhm_sample)
    max_peak.append(w[np.argmax(d)])




In [5]:
hwhm = np.array(hwhm)
mean_median_metric = np.array(mean_median_metric)
outputs = normalization(hwhm) + normalization(mean_median_metric)
outputs  = 1/ outputs
outputs = normalization(outputs)
outputs_peak = np.array(max_peak)


# remove double peaks

In [6]:

inputs_pruned = inputs.copy()
outputs_pruned = outputs.copy()
outputs_peak_pruned = outputs_peak.copy()


indices_of_peaks = np.arange(0,file_nr)
indices_of_peaks_pruned = indices_of_peaks.copy()

double_peak_samples = [220,219,216,215,214,209,182,180,179,177,174,173,172,170,167, 166,165,163,161,160,159,158,157,156, 153,152,149,143,137,134,126,125,124,123,121,117,115,105,104,103,87,69,39,38,22,18]
double_peak_samples = [x - 1 for x in double_peak_samples]
# print(double_peak_samples)
for i in double_peak_samples :
    indices_of_peaks_pruned = np.delete(indices_of_peaks_pruned, i, axis=0)
    inputs_pruned = np.delete(inputs_pruned, i, axis=0)
    outputs_pruned = np.delete(outputs_pruned, i, axis=0)
    outputs_peak_pruned = np.delete(outputs_peak_pruned, i, axis=0)

print(inputs_pruned.shape)
print(outputs_pruned.shape)
print(outputs_peak_pruned.shape)



(174, 3)
(174, 1)
(174, 1)


# Peak Position Prediction NN

In [ ]:
def model_peak (X_train): #X_train is our training input

    inp_1 = Input(shape=(X_train.shape[1],))  #setting the size of the input layer
    initial = 'he_uniform'
    x = Dense(5,kernel_initializer= initial, activation = 'elu', bias_initializer=initial)(inp_1)
    x = Dense(5,kernel_initializer= initial, activation = 'relu', bias_initializer=initial)(x)
    x = Dense(5,kernel_initializer= initial, activation = 'relu', bias_initializer=initial)(x)
    x = Dense(1,kernel_initializer= initial,  activation = 'elu',bias_initializer=initial)(x)

    return Model(inputs=inp_1, outputs=x)

In [ ]:

lr_schedule = tfk.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=100,
    decay_rate=0.99)    

predictions_test_batch = []
y_true_batch = []
for i in range(0,30):
    X_train_peak, X_test_peak, y_train_peak, y_test_peak = model_selection.train_test_split(inputs_pruned,outputs_peak_pruned, test_size=0.02,random_state = i ) 

    opt = tfk.optimizers.RMSprop(learning_rate=lr_schedule,clipvalue=0.8)
    #model.summary()
    peak_model = Sequential()  # instantiate the model 
    peak_model = model_peak(X_train_peak)  #give it the parameters given in the function model_dam



    peak_model.compile(loss = customLoss, optimizer= opt, metrics=[customLoss]) #compile the model 


    checkpoint_filepath = '/tmp/checkpoint'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=False,
        monitor='val_loss',
        mode='min',
        save_best_only=True)


    history_callback = peak_model.fit(X_train_peak, y_train_peak,validation_data=(X_test_peak, y_test_peak),epochs=1000, verbose = 0)

    predictions_test_batch.append(peak_model.predict(X_test_peak))
    y_true_batch.append(y_test_peak)

#     print(i)
y_true_batch = np.array(y_true_batch)
predictions_test_batch = np.array(predictions_test_batch)
predictions_test_batch = np.reshape(predictions_test_batch, (predictions_test_batch.shape[0]*predictions_test_batch.shape[1],1))
y_true_batch = np.reshape(y_true_batch, (y_true_batch.shape[0]*y_true_batch.shape[1],1))

print("median error % ", np.median(100*abs((predictions_test_batch-y_true_batch)/y_true_batch)) )



# Classification of formability

In [ ]:
print(inputs.shape)
double_peak_indicator = np.zeros((inputs.shape[0],1))
for i in double_peak_samples:
    double_peak_indicator[i] = 1

In [ ]:

skf = StratifiedKFold(n_splits = 5, random_state=seed_value, shuffle=True)
skf.get_n_splits(inputs, double_peak_indicator)
dp_pred = []
dp_true = []
print(skf)

for train_index, test_index in skf.split(inputs, double_peak_indicator):
    double_peak_RF = RandomForestClassifier(max_depth = 150,min_samples_split = 10, random_state= 42)
    input_for_double_train, input_for_double_test = inputs[train_index], inputs[test_index]
    double_peak_train, double_peak_test = double_peak_indicator[train_index], double_peak_indicator[test_index]
    
    double_peak_RF.fit(input_for_double_train, double_peak_train)

    prediction = double_peak_RF.predict(input_for_double_test)
    prediction = np.array(prediction)
    prediction = np.reshape(prediction, (prediction.shape[0],1))
    try:
        dp_pred = np.vstack((dp_pred,prediction ))
    except:
        dp_pred = prediction
    try:
        dp_true = np.vstack((dp_true,double_peak_test ))
    except:
        dp_true = double_peak_test
    
dp_pred = np.array(dp_pred)   
dp_true = np.array(dp_true)   

dp_pred = np.reshape(dp_pred, (dp_pred.shape[0],1))
cnf_matrix = confusion_matrix(dp_true,dp_pred)
print()
print("random forest  ")
print("error_rf = ", 100*len(np.nonzero(dp_pred-dp_true)[0])/dp_true.shape[0] )
print("confusion matrix")
print(cnf_matrix)

print()

# Objective Function Prediction

In [ ]:
pred = []
truth = []
std = []


sigma_f, l = 0.5,1
for i in range(30):

    X_train, X_test, y_train, y_test = model_selection.train_test_split(inputs_pruned,outputs_pruned, test_size=0.02,random_state = i )

    kernel = GPy.kern.RBF(1, sigma_f, l)
    kernel += GPy.kern.Matern32(input_dim=inputs_pruned.shape[1])
    
    model = GPy.models.GPRegression(X_train,y_train,kernel) 
    model.optimize()
    mean, variance = model.predict(X_test)
    
    pred.append(mean)
    std.append(100*np.divide(variance, y_test))
    truth.append(y_test)
#     print(i)


truth = np.array(truth)
pred = np.array(pred)
pred = np.reshape(pred, (pred.shape[0]*pred.shape[1],1))
pred = np.maximum(0,pred)
truth = np.reshape(truth, (truth.shape[0]*truth.shape[1],1))
print("median % relative error ", np.median(100*abs((pred-truth)/truth)) )



# train all the models

In [ ]:
peak_model = Sequential()  # instantiate the model 
peak_model = model_peak(inputs)  #give it the parameters given in the function model_dam

lr_schedule = tfk.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=100,
    decay_rate=0.99)    


    
    
opt = tfk.optimizers.RMSprop(learning_rate=lr_schedule,clipvalue=0.8)
peak_model.compile(loss = customLoss, optimizer= opt, metrics=[customLoss]) #compile the model 

peak_model.fit(inputs_pruned, outputs_peak_pruned,epochs=1000, verbose = 0)

sigma_f, l = 0.1,0.2
kernel = GPy.kern.RBF(1, sigma_f, l) + GPy.kern.Matern32(input_dim=inputs_pruned.shape[1])
model_shallow = GPy.models.GPRegression(inputs_pruned, outputs_pruned,kernel) 
model_shallow.optimize()


double_peak_indicator = np.zeros((inputs.shape[0],1))
for i in double_peak_samples:
    double_peak_indicator[i] = 1

double_peak_RF = RandomForestClassifier(max_depth=50, random_state= seed_value)
double_peak_RF.fit(inputs, np.ravel(double_peak_indicator))



print("trained")


# Identify best sample so far

In [ ]:
def lookup(nanoplates = 2):
    nanoplates = nanoplates - 2
    nanoplates = np.minimum(np.maximum(0,nanoplates),9)
    lookup_table = [[427,438,435],[455,467,460],[472,479,477],[484,489,487],[491,498,496],[499,503,501],[504,507,505],[510,525,515]]
    hard_lower, hard_upper, soft_middle = lookup_table[nanoplates]
    return hard_lower, hard_upper, soft_middle


def optimum_for_nanoplatelet(x, model_objective,outputs_peak_pruned , nanoplatelets = 2):
    best_so_far = 0
    lookup_table = [[427,438,435],[455,467,460],[472,479,477],[484,489,487],[491,498,496],[499,503,501],[504,507,505],[510,525,515]]

    for i in range( outputs_peak_pruned.shape[0]):
        if outputs_peak_pruned[i] >=lookup(nanoplatelets)[0] and outputs_peak_pruned[i] <=lookup(nanoplatelets)[1] :
            pred_temp,_ = model_objective.predict(x[i:i+1])
            if pred_temp > best_so_far:
                best_so_far = pred_temp
    return best_so_far[0][0]

# BO 

In [ ]:

def expected_improvement(X, optimal_so_far, gpr, tradeoff=0.01):
    mu, sigma = gpr.predict(X)
    sigma = sigma.reshape(-1, 1)
    mu_sample_opt = optimal_so_far
    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - tradeoff
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    return ei


def get_score(Xsamples,optimal_so_far, model,peak_prediction, middle_constraints, experimental_constraints,method = "pe", nanoplate = 2,tradeoff = 0.5 ):
    lower,upper,middle = lookup(nanoplate)
    mu, std = model.predict(Xsamples)
    mu = mu[:, 0]
    std = std[:,0]
    predicted_peak  = peak_prediction.predict(Xsamples)
    std = np.reshape(std, (std.shape[0],1))
    distance_from_inputs = np.ones((mu.shape[0],1))
    mu = np.reshape(mu, (mu.shape[0],1))
    if middle_constraints == 1:
        constraint_middle = np.multiply(np.divide(abs(middle-predicted_peak),(upper-lower)),mu)

    else:
        constraint_middle = 0

    predicted_feasibility = double_peak_RF.predict(Xsamples)
    
    
    
    constrain_formability = np.exp(predicted_feasibility*sys.maxsize) - 1 
    constrain_formability = np.reshape(constrain_formability, (constrain_formability.shape[0],1))
    

    constraints_terms = np.exp(sys.maxsize*( lower - predicted_peak)) +  np.exp(sys.maxsize*( predicted_peak - upper)) + constraint_middle + constrain_formability
    if method == "pi":

        scores = norm.cdf((mu - optimal_so_far) / (std+1E-9)) -constraints_terms  

    elif method == "pe":
        scores = expected_improvement(Xsamples,optimal_so_far, model, tradeoff)  - constraints_terms
        
    elif method == "ucb":
        scores = mu + tradeoff*std - constraints_terms
    return scores

def get_suggestions(optimal_so_far, model, peak_prediction, lower_bound,middle_constraints = 1,experimental_constraints = 0,method = "pe", nanoplate = 2,tradeoff = 0.5):
    Xsamples = []
    sample_size = 100000
    if experimental_constraints == 1:
        acetone_constrained = (np.random.uniform(low=lower_bound, high=0.9-min_cons, size=(sample_size,1)))
        b_temp = (np.random.uniform(low= lower_bound, high= 1-acetone_constrained-lower_bound, size=(sample_size,1)))
        c_temp = 1 - acetone_constrained - b_temp
        test = acetone_constrained + b_temp + c_temp

        PbBr2_constrained = np.maximum(b_temp, c_temp)
        oleate_constrained = np.minimum(b_temp, c_temp)
        Xsamples = np.array(np.hstack((PbBr2_constrained,oleate_constrained,acetone_constrained)))
    else:
    
        Xsamples = np.random.uniform(size=(sample_size,3))
        summation = np.sum(Xsamples, axis = 1)
        summation  = np.reshape(summation, (summation.shape[0],1))
        Xsamples = Xsamples/ summation
    scores = get_score(Xsamples,optimal_so_far, model,peak_prediction, middle_constraints,experimental_constraints, method,nanoplate, tradeoff)

    ix = np.argmax(scores)
    iy = np.argmin(scores)


    if math.isinf((scores[ix])):
        return np.array([0,0,0])
    else:
        return Xsamples[ix]

# RUN BO

In [ ]:
acquisition_values = []
experimental_constraints = 0
tradeoff = 0.01  # higher tradeoff = more exploration 
for middle_const in [0]:
    for method in ["pe"]:
        for lower_bound in [0]:
            print("middle constraitns = ", middle_const, " method = ", method)
            print()
            for nano in [2,3,4,5,6,7,8,9]:
                optimal_so_far = optimum_for_nanoplatelet(inputs_pruned,model_shallow, outputs_peak_pruned,nano)

                if nano == 9:
                    print("      nanocubes")
                else:
                    print('      number of nanoplates : ',nano)
                best_mean_shallow = 0
                best_std_shallow = 0
                for i in range(10): 
                    best = get_suggestions(optimal_so_far, model_shallow,peak_model,lower_bound,middle_const,experimental_constraints,method,nano, tradeoff)
                    best = np.reshape(best, (1,best.shape[0]))
                    results = model_shallow.predict(best)
                    acquisition_values.append(best)
                    mean_shallow = results[0][0][0]
                    std_shallow = results[1][0][0]
                    if mean_shallow > best_mean_shallow:
                        sample_mean_shallow = best
                        best_mean_shallow = mean_shallow
                    if std_shallow > best_std_shallow:
                        best_std_shallow = std_shallow
                        sample_std_shallow = best
                print("           spectrum range ",lookup(nano))
                print("           composition ", sample_mean_shallow)
                print("           objective function ", model_shallow.predict(sample_mean_shallow)[0][0][0], 'uncertainty ', model_shallow.predict(sample_mean_shallow)[1][0][0])
                print("           predicted peak ",peak_model.predict(sample_mean_shallow)[0][0])
                print("           optimal so far ", optimal_so_far)
                print()